<a href="https://colab.research.google.com/github/giorgiosld/Natural-Language-Processing/blob/main/lab4/T_725_Lab04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# T-725 Natural Language Processing: Lab 4
In today's lab, we will be working with part-of-speech tagging, parsing, and chunking.

To begin with, do the following:
* Select `"File" > "Save a copy in Drive"` to create a local copy of this notebook that you can edit.
* Select `"Runtime" > "Run all"` to run the code in this notebook.

## Setting up
Let's start off by installing the `svgling` library for drawing syntax trees, as well as defining a function to evaluate the grammars that we will be creating in today's lab assignment.

In [ ]:
!pip install svgling

In [ ]:
# Compare trees generated by a parser to correct trees

def evaluate_parser(parser, sentences, correct):
  correct_trees = [nltk.Tree.fromstring(t) for t in correct]
  num_sentences = len(sentences)

  for num, (sent, correct_tree) in enumerate(zip(sentences, correct_trees)):
    print(f">{sent}\n")

    error = None
    print_tree = False

    try:
      my_trees = list(parser.parse(sent.split()))
    except ValueError as e:
      print(e)
      my_trees = []

    if len(my_trees) > 1:
      error = f"Generated {len(my_trees)} trees (should only generate one)!"
    elif len(my_trees) == 0:
      error = "Couldn't parse sentence."
    else:
      my_tree = my_trees[0]
      if my_tree != correct_tree:
        error = "Generated an incorrect tree"
        print_tree = True

    if error:
      print(error)

      if print_tree:
        print("\nYour tree:")
        my_tree.pretty_print()

      print("\nCorrect tree:")
      correct_tree.pretty_print()
    else:
      print("Correct!\n")

    if num != num_sentences - 1:
      print("============================")

## Context free grammars (CFG) in NLTK

[Example 3.1](https://www.nltk.org/book/ch08.html#sec-context-free-grammar) in chapter 8 of the NLTK book shows how we define a context free grammar (CFG) and use it to parse sentences.

In [ ]:
import nltk

grammar1 = nltk.CFG.fromstring("""
  S -> NP VP
  VP -> V NP | V NP PP
  PP -> P NP
  V -> "saw" | "ate" | "walked"
  NP -> "John" | "Mary" | "Bob" | Det N | Det N PP
  Det -> "a" | "an" | "the" | "my"
  N -> "man" | "dog" | "cat" | "telescope" | "park"
  P -> "in" | "on" | "by" | "with"
  """)

sent = "Mary saw Bob".split()
rd_parser = nltk.RecursiveDescentParser(grammar1)
for tree in rd_parser.parse(sent):
  print(tree)

## Visualizing parse trees with NLTK
We can use NLTK to draw a graphical diagram of a syntax tree. Unfortunately, this functionality does not work on Google Colab. We can still print it as text:

In [ ]:
from nltk import Tree
t = Tree.fromstring('(S (NP this tree) (VP (V is) (ADJ pretty)))')

# NLTK won't draw the tree if we're using Google Colab
# t.draw()

# But we can still print a text diagram
t.pretty_print()

We can also use a different library, `svgling`, to draw trees:

In [ ]:
import svgling

svgling.draw_tree(t)

## Probabilistic context free grammar (PCFG) in NLTK
[Example 6.4](https://www.nltk.org/book/ch08.html#code-pcfg1) in chapter 8 of the NLTK book shows how to define a probabilistic context free grammar (PCFG) and use it to produce the most likely tree for a given sentence. Note that the productions for each category must add up to 1.0.

In [ ]:
import nltk

grammar = nltk.PCFG.fromstring("""
    S    -> NP VP              [1.0]
    VP   -> TV NP              [0.4]
    VP   -> IV                 [0.3]
    VP   -> DatV NP NP         [0.3]
    TV   -> 'saw'              [1.0]
    IV   -> 'ate'              [1.0]
    DatV -> 'gave'             [1.0]
    NP   -> 'telescopes'       [0.8]
    NP   -> 'Jack'             [0.2]
    """)

viterbi_parser = nltk.ViterbiParser(grammar)

for tree in viterbi_parser.parse(['Jack', 'saw', 'telescopes']):
  print(tree)

This is equivalent to:

In [ ]:
grammar = nltk.PCFG.fromstring("""
    S    -> NP VP [1.0]
    VP   -> TV NP [0.4] | IV [0.3] | DatV NP NP [0.3]
    TV   -> 'saw' [1.0]
    IV   -> 'ate' [1.0]
    DatV -> 'gave' [1.0]
    NP   -> 'telescopes' [0.8] | 'Jack' [0.2]
    """)

viterbi_parser = nltk.ViterbiParser(grammar)

for tree in viterbi_parser.parse(['Jack', 'saw', 'telescopes']):
  print(tree)

## Chunking with NLTK
NLTK's taggers can be used for more than just assigning part-of-speech tags to tokens. They can also be trained to assign chunk tags to chunks. [Example 3.1](https://www.nltk.org/book/ch07.html#code-unigram-chunker) in chapter 7 of the NLTK book shows how to build a chunker with a unigram tagger:

In [ ]:
class UnigramChunker(nltk.ChunkParserI):
    def __init__(self, train_sents):
        train_data = [[(t, c) for w, t, c in nltk.chunk.tree2conlltags(sent)]
                      for sent in train_sents]

        self.tagger = nltk.UnigramTagger(train_data)

    def parse(self, sentence):
        pos_tags = [pos for (word,pos) in sentence]
        tagged_pos_tags = self.tagger.tag(pos_tags)
        chunktags = [chunktag for (pos, chunktag) in tagged_pos_tags]
        conlltags = [(word, pos, chunktag) for ((word,pos),chunktag)
                     in zip(sentence, chunktags)]
        return nltk.chunk.conlltags2tree(conlltags)

We can train it and evaluate it on the CoNLL 2000 corpus, which contains chunk IOB tags (I-inside, O-outside, B-begin) as well as POS tags.

In [ ]:
from nltk.corpus import conll2000
nltk.download('conll2000')
nltk.download('averaged_perceptron_tagger')

test_sents = conll2000.chunked_sents('test.txt', chunk_types=['NP'])
train_sents = conll2000.chunked_sents('train.txt', chunk_types=['NP'])

chunker = UnigramChunker(train_sents)

Now that we've trained an NP chunker, we can try chunking a sentence:

In [ ]:
import svgling
sentence = nltk.pos_tag('The little yellow dog barked at the cat.'.split())

tree = chunker.parse(sentence)
print(tree)

# Note that we can't pretty_print() partial trees, but we can still draw them
# using svgling
svgling.draw_tree(tree)

We can also see how well the chunker performs by evaluating it on the test set:

In [ ]:
print(chunker.evaluate(test_sents))

# Assignment
Answer the following questions and hand in your solution in Canvas before 8:30 Monday morning, September 25th. Remember to save your file before uploading it.

## Question 1
Part-of-Speech Tagging

Complete the following tasks:
1. Use `gutenberg.sents(filename)` to get the tokenized sentences of *The Innocence of Father Brown* by G. K. Chesterton (`chesterton-brown.txt`).
2. Tag the sentences using `nltk.pos_tag_sents(tokenized_sentences)`.
3. Use a list comprehension to create a single list with all the tags.
4. Use `nltk.FreqDist()` to count the tags, and generate a frequency list (with all the tags) using `.most_common()`.


In [ ]:
from nltk.corpus import gutenberg
nltk.download('gutenberg')

# Required for nltk.pos_tag_sents()
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')

In [ ]:
# Your solution here


## Question 2
Add to `q1_grammar` below until the sentence can be correctly parsed.

In [ ]:
q1_grammar = nltk.CFG.fromstring("""
  S  -> NP VP
  """)

q1_parser = nltk.ChartParser(q1_grammar)

# Evaluate the parser
q1_sents = ["sunsets on mars are blue"]
q1_correct = ["(S (NP (NP (N sunsets)) (PP (P on) (NP (N mars)))) (VP (V are) (ADJ blue)))"]

evaluate_parser(q1_parser, q1_sents, q1_correct)

## Question 3
Make the following changes to `q2_grammar` below:
1. Replace the noun category (N) with singular nouns (NN) and plural nouns (NNS).
2. Add a category for cardinal numbers (CD).
3. Add a category for adverbs (ADV).
4. Add a category for personal pronouns (PRP).

Adjust your grammar so that the sentences below are parsed correctly.

In [ ]:
q2_grammar = nltk.CFG.fromstring("""
  S  -> NP VP
  NP -> NP PP | N | DT ADJ N
  VP -> V NP
  PP -> P NP
  DT -> 'the'
  N -> 'venus' | 'planet' | 'earth'
  ADJ -> 'closest'
  V -> 'is'
  P -> 'to'
  """)

q2_parser = nltk.ChartParser(q2_grammar)

# Facts
q2_facts = [
    "venus is the closest planet to earth",
    "the sun is halfway through its life",
    "saturn has 82 known moons"
]

# Evaluate the parser
q2_correct = [
    "(S (NP (NN venus)) (VP (V is) (NP (NP (DT the) (ADJ closest) (NN planet)) (PP (P to) (NP (NN earth))))))",
    "(S (NP (DT the) (NN sun)) (VP (V is) (PP (ADV halfway) (P through) (NP (PRP its) (NN life)))))",
    "(S (NP (NN saturn)) (VP (V has) (NP (CD 82) (ADJ known) (NNS moons))))"
]

evaluate_parser(q2_parser, q2_facts, q2_correct)

## Question 4
Replace the verb category (V) with transitive (TV) and intransitive (IV) verbs and adjust the grammar so that the sentences below are correctly parsed.

In [ ]:
q3_grammar = nltk.CFG.fromstring("""
  S  -> NP VP
  NP -> DT N N | N | CD
  VP -> V PP PP | V NP PP
  PP -> P | P NP
  DT -> 'the'
  N -> 'curiosity' | 'rover' | 'mars' | 'nasa' | 'perseverence'
  V -> 'arrived' | 'launched'
  P -> 'on' | 'in'
  CD -> '2012' | '2020'
  """)

q3_parser = nltk.ChartParser(q3_grammar)

# Facts
q3_facts = [
    "the curiosity rover arrived on mars in 2012",
    "nasa launched the perseverence rover in 2020"
]

# Evaluate the parser
q3_correct = [
    "(S (NP (DT the) (N curiosity) (N rover)) (VP (IV arrived) (PP (P on) (NP (N mars))) (PP (P in) (NP (CD 2012)))))",
    "(S (NP (N nasa)) (VP (TV launched) (NP (DT the) (N perseverence) (N rover)) (PP (P in) (NP (CD 2020)))))"
]

evaluate_parser(q3_parser, q3_facts, q3_correct)

## Question 5
The following grammar produces two trees for the sentence *John saw a man with binoculars*. Each tree represents a different interpretation of the sentence. One tree implies that John used binoculars to see a man, while the other implies that a man with binoculars was seen by John.

Change the grammar into a **probabilistic CFG** and replace the `ChartParser` with a `ViterbiParser`, which only outputs the most probable tree. Adjust the probabilities so that the parser produces the tree that implies that John saw a man who had binoculars.

In [ ]:
q1_grammar = nltk.CFG.fromstring("""
  S -> NP VP
  NP -> N | NP PP | DT N
  VP -> V NP | V NP PP
  PP -> P NP
  V -> "saw"
  DT -> "a"
  N -> "John" | "man" | "binoculars"
  P -> "with"
  """)

sentence = "John saw a man with binoculars".split()

q1_parser = nltk.ChartParser(q1_grammar)
trees = q1_parser.parse(sentence)

for tree in trees:
  print(tree)
  tree.pretty_print()
  print()

## Question 6
Create a new class, `TrigramChunker`, based on the `UnigramChunker` class defined earlier in the notebook. Modify it so that `self.tagger` is a trigram tagger with a unigram tagger as a backoff (see [5.4 Combining taggers](http://www.nltk.org/book/ch05.html#combining-taggers) in the NLTK book). Train and evaluate your chunker on the CoNLL 2000 corpus (using `train_sents` and `test_sents`, created earlier) and report the results.

In [ ]:
# Your solution here


## Question 7
Complete the following tasks:

1. Create a list of all tagged sentences from `test.txt` in the `conll2000` corpus using its `tagged_sents(filename)` method.

2. This should return a list of 2012 tagged sentences. You can access a sentence from the list by its index, e.g., `tagged_sents_list[0]` will return the first tagged sentence. Parse any three of the taggged sentences with your chunker and draw the trees using `svgling.draw_tree(tree)`. Note that the tree will only be drawn if the `draw_tree` method is called at the end of the code block.

3. Compare the trees generated by your chunker to the correct trees in `test_sents` (which contains the same sentences in the same order as those returned by the `tagged_sents` method). Does your chunker make any mistakes? If so, give an example of at least one of those mistakes.

In [ ]:
# Your solution here


In [ ]:
# Sentence 1


In [ ]:
# Sentence 2


In [ ]:
# Sentence 3
